### Installation

In [ ]:
!pip install -q unsloth
# Also get the latest nightly Unsloth!
!pip install -q --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
!pip install google-generativeai gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.3/351.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 32.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.5/170.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 105.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

### Unsloth

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)



*   **model_name:** Specifies the name of the pre-trained model to load.
*   **max_seq_length:** Defines the maximum sequence length (in tokens) that the model can process. max_seq_length = 2048 allows the model to process sequences up to 2048 tokens long.
*   **dtype:** Specifies the data type for model weights and computations. None: Automatically selects the appropriate data type based on the hardware. torch.float16: Uses 16-bit floating point precision, reducing memory usage and potentially increasing speed on compatible GPUs. torch.bfloat16: Similar to float16 but with a wider dynamic range, beneficial for certain hardware like NVIDIA A100 GPUs.
*   **load_in_4bit:** Determines whether to load the model using 4-bit quantization.Ideal for scenarios where memory efficiency is crucial, such as deploying models on edge devices or during experimentation.





In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16, # a higher alpha value assigns more weight to the LoRA activations
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)



*   **r:** The rank of the low-rank matrices in LoRA; higher values can capture more information but increase memory usage.
*   **target_modules:** List of model components (e.g., "q_proj", "k_proj") where LoRA adapters are inserted for fine-tuning.
*   **lora_alpha:** Scaling factor for the LoRA updates; controls the impact of the adapters on the model's outputs.
*   **lora_dropout:**  Dropout rate applied to LoRA layers during training to prevent overfitting.
*   **bias:**   Specifies how biases are handled in LoRA layers; options include "none", "all", or "lora_only".
*   **use_gradient_checkpointing:**    Enables gradient checkpointing to reduce memory usage during training; "unsloth" uses Unsloth's optimized version.
*   **random_state:**   Seed for random number generators to ensure reproducibility of training results.
*   **use_rslora:**     Boolean indicating whether to use Rank-Stabilized LoRA (rsLoRA) for potentially more stable training.
*   **loftq_config:**   Configuration for Low-Rank Quantization (LoftQ); set to None to disable this feature.


### Data Prep

In [ ]:
from datasets import Dataset

rows = [
    # --- Education & Opportunity ---
    {"instruction": "Decide if the story uplifts humanity and explain why.",
     "input": "A community organizes free coding classes for kids.",
     "output": "Yes. It broadens access to skills and future opportunity while fostering mentorship.",
     "score": 0.95, "category": "education", "tags": ["youth","access","skills"]},

    {"instruction": "Is this uplifting for humanity? Provide a short rationale.",
     "input": "A wealthy school district bans sharing textbooks with neighboring underfunded schools.",
     "output": "No. It entrenches inequality and withholds knowledge that could elevate others.",
     "score": 0.05, "category": "education", "tags": ["inequality","gatekeeping"]},

    {"instruction": "Rate 0–1 how much this uplifts humanity, then justify.",
     "input": "A nonprofit funds bilingual libraries in rural towns.",
     "output": "0.9 — It expands literacy and cross-cultural understanding.",
     "score": 0.90, "category": "education", "tags": ["literacy","inclusion","language"]},

    {"instruction": "Classify as uplifting / not / mixed and explain.",
     "input": "An elite scholarship requires unpaid labor for the sponsor’s private estate.",
     "output": "Mixed. It creates access but exploits labor and undermines dignity.",
     "score": 0.45, "category": "education", "tags": ["access","exploitation"]},

    # --- Health & Mental Health ---
    {"instruction": "Does this uplift humanity? Explain briefly.",
     "input": "Volunteers create a crisis text line staffed by trained listeners at all hours.",
     "output": "Yes. It provides compassionate support and reduces isolation during critical moments.",
     "score": 0.93, "category": "mental_health", "tags": ["suicide_prevention","community_care"]},

    {"instruction": "Evaluate for uplifting-humanity and justify.",
     "input": "A clinic denies treatment based on a patient's immigration status.",
     "output": "No. It devalues human life and erodes social trust and compassion.",
     "score": 0.02, "category": "healthcare", "tags": ["exclusion","ethics"]},

    {"instruction": "Rate 0–1 and give a rationale.",
     "input": "A biotech startup open-publishes a low-cost insulin synthesis guide for LMICs.",
     "output": "0.95 — It advances access to lifesaving medicine and invites global collaboration.",
     "score": 0.95, "category": "healthcare", "tags": ["access_medicines","open_knowledge"]},

    {"instruction": "Decide uplift and explain reasoning.",
     "input": "A hospital replaces nurses with untrained volunteers to cut costs.",
     "output": "No. It compromises safety and disrespects professional care standards.",
     "score": 0.08, "category": "healthcare", "tags": ["safety","labor_rights"]},

    # --- Environment & Climate ---
    {"instruction": "Is this uplifting? Why or why not?",
     "input": "City residents convert vacant lots into community gardens.",
     "output": "Yes. It improves food access, strengthens community, and reconnects people with nature.",
     "score": 0.92, "category": "environment", "tags": ["food_security","community","green_space"]},

    {"instruction": "Evaluate uplift with a justification.",
     "input": "A documentary glamorizes eco-terrorism as the only solution.",
     "output": "No. It promotes harm and alienates constructive collective action.",
     "score": 0.10, "category": "environment", "tags": ["violence","polarization"]},

    {"instruction": "Rate from 0 to 1 and explain.",
     "input": "A company commits to science-based net-zero targets and open-sources its decarbonization playbook.",
     "output": "0.85 — It models accountability and accelerates wider emissions reduction.",
     "score": 0.85, "category": "climate", "tags": ["decarbonization","open_playbook"]},

    {"instruction": "Binary decision with rationale.",
     "input": "A tourism campaign encourages visitors to litter less but sells single-use plastics everywhere.",
     "output": "Mixed. The message helps, but actions contradict it and blunt impact.",
     "score": 0.4, "category": "environment", "tags": ["hypocrisy","behavior_change"]},

    # --- Technology & AI ---
    {"instruction": "Does this scenario uplift humanity? Provide reasoning.",
     "input": "An open-source AI accessibility tool reads websites aloud for visually impaired users.",
     "output": "Yes. It expands autonomy and inclusion through shared technology.",
     "score": 0.96, "category": "technology", "tags": ["accessibility","open_source","inclusion"]},

    {"instruction": "Assess uplifting value and justify.",
     "input": "A social app optimizes for outrage to boost engagement.",
     "output": "No. It corrodes empathy and amplifies division.",
     "score": 0.03, "category": "media", "tags": ["outrage","polarization","ad_optimization"]},

    {"instruction": "Rate 0–1; explain succinctly.",
     "input": "Researchers release a dataset of faces without consent for surveillance training.",
     "output": "0.0 — Violates privacy and dignity, enabling abuse.",
     "score": 0.00, "category": "AI", "tags": ["privacy","consent","surveillance"]},

    {"instruction": "Classify and explain.",
     "input": "A civic hackathon builds an offline-first disaster alerts app for rural areas.",
     "output": "Yes. It improves resilience and equitable access to life-saving info.",
     "score": 0.9, "category": "technology", "tags": ["resilience","offline_first","public_good"]},

    # --- Arts, Media, & Culture ---
    {"instruction": "Is this uplifting? Explain briefly.",
     "input": "A film explores grief and reconciliation, helping audiences process loss.",
     "output": "Yes. It deepens empathy and offers pathways to healing.",
     "score": 0.88, "category": "arts", "tags": ["empathy","healing"]},

    {"instruction": "Judge uplift and provide a reason.",
     "input": "A reality show humiliates contestants for entertainment.",
     "output": "No. It normalizes cruelty and dehumanization.",
     "score": 0.06, "category": "media", "tags": ["humiliation","cruelty"]},

    {"instruction": "Rate 0–1 and justify.",
     "input": "A music festival dedicates a portion of tickets to low-income youth and funds music education.",
     "output": "0.8 — It expands cultural access and reinvests in future creativity.",
     "score": 0.80, "category": "arts", "tags": ["access","youth","reinvestment"]},

    {"instruction": "Binary decision + rationale.",
     "input": "A popular influencer spreads conspiracies to grow their audience.",
     "output": "No. It undermines shared reality and civic trust.",
     "score": 0.04, "category": "media", "tags": ["misinformation","trust_erosion"]},

    # --- Community & Solidarity ---
    {"instruction": "Does this uplift humanity? Explain.",
     "input": "Neighbors coordinate meal trains for families in crisis.",
     "output": "Yes. It builds mutual aid and reduces suffering.",
     "score": 0.94, "category": "community", "tags": ["mutual_aid","care"]},

    {"instruction": "Evaluate uplift.",
     "input": "A nonprofit pits groups against each other to compete for basic services funding.",
     "output": "No. It fosters scarcity thinking and division.",
     "score": 0.12, "category": "community", "tags": ["division","scarcity_mindset"]},

    {"instruction": "Score 0–1 with reason.",
     "input": "A local news co-op trains residents to report solutions journalism.",
     "output": "0.85 — It amplifies constructive narratives and civic agency.",
     "score": 0.85, "category": "media", "tags": ["solutions_journalism","civic_agency"]},

    {"instruction": "Classify uplift status and explain.",
     "input": "A city installs hostile architecture to deter people experiencing homelessness.",
     "output": "No. It degrades dignity and avoids humane solutions.",
     "score": 0.01, "category": "housing", "tags": ["dignity","hostile_architecture"]},

    # --- Governance & Justice ---
    {"instruction": "Is this uplifting? Provide rationale.",
     "input": "A jurisdiction replaces cash bail with risk assessments and support services.",
     "output": "Yes. It reduces inequity and supports fairer outcomes.",
     "score": 0.82, "category": "justice", "tags": ["equity","reform"]},

    {"instruction": "Assess uplift level.",
     "input": "A politician stokes fear against minorities to win votes.",
     "output": "No. It weaponizes division and harms vulnerable groups.",
     "score": 0.0, "category": "governance", "tags": ["demagoguery","harm"]},

    {"instruction": "Rate 0–1; justify.",
     "input": "A city publishes open budgets with participatory input from residents.",
     "output": "0.88 — Transparency and shared decision-making strengthen democracy.",
     "score": 0.88, "category": "governance", "tags": ["transparency","participation"]},

    {"instruction": "Binary verdict + reason.",
     "input": "A law criminalizes peaceful protest near government buildings.",
     "output": "No. It chills speech and weakens civic engagement.",
     "score": 0.07, "category": "rights", "tags": ["free_expression","assembly"]},

    # --- Economy & Labor ---
    {"instruction": "Does this uplift humanity? Explain briefly.",
     "input": "A platform co-op shares profits and decisions with its workers.",
     "output": "Yes. It distributes power and rewards contribution more fairly.",
     "score": 0.86, "category": "economy", "tags": ["cooperative","shared_ownership"]},

    {"instruction": "Evaluate uplift.",
     "input": "A firm uses noncompete clauses to trap low-wage workers.",
     "output": "No. It restricts mobility and entrenches exploitation.",
     "score": 0.05, "category": "labor", "tags": ["noncompete","exploitation"]},

    {"instruction": "Rate 0–1 and justify.",
     "input": "An app enables transparent salary data to close pay gaps.",
     "output": "0.78 — Informs fair negotiation and reduces inequity.",
     "score": 0.78, "category": "economy", "tags": ["pay_transparency","equity"]},

    {"instruction": "Classify and explain.",
     "input": "Gig workers are labeled contractors to avoid safety nets.",
     "output": "No. It externalizes risk and undermines dignity.",
     "score": 0.1, "category": "labor", "tags": ["gig_economy","precarity"]},

    # --- Science & Space ---
    {"instruction": "Is this uplifting? Why?",
     "input": "An international team open-shares a climate model that local planners can adapt.",
     "output": "Yes. It accelerates practical solutions through shared science.",
     "score": 0.9, "category": "science", "tags": ["open_science","adaptation"]},

    {"instruction": "Evaluate uplifting value.",
     "input": "A private space venture dumps deorbited debris into fragile ecosystems.",
     "output": "No. It prioritizes profit over planetary stewardship.",
     "score": 0.08, "category": "space", "tags": ["environmental_harm","negligence"]},

    {"instruction": "Rate 0–1 and justify.",
     "input": "A university adopts diamond open access for publicly funded research.",
     "output": "0.9 — Freely accessible knowledge expands global learning and innovation.",
     "score": 0.9, "category": "science", "tags": ["open_access","public_good"]},

    {"instruction": "Binary verdict + reason.",
     "input": "A lab suppresses negative results that would prevent harm.",
     "output": "No. It undermines integrity and public safety.",
     "score": 0.07, "category": "science", "tags": ["research_integrity","safety"]},

    # --- Equity & Human Rights ---
    {"instruction": "Does this scenario uplift humanity? Explain.",
     "input": "A theater installs captions and relaxed performances for neurodiverse audiences.",
     "output": "Yes. It removes barriers and invites full participation.",
     "score": 0.91, "category": "accessibility", "tags": ["captions","neurodiversity","inclusion"]},

    {"instruction": "Assess uplift.",
     "input": "A company forbids speaking non-dominant languages at work.",
     "output": "No. It stigmatizes identity and erodes belonging.",
     "score": 0.03, "category": "equity", "tags": ["language_rights","belonging"]},

    {"instruction": "Rate 0–1; provide rationale.",
     "input": "A mentorship network pairs first-gen college students with professionals.",
     "output": "0.83 — Builds social capital and opens doors.",
     "score": 0.83, "category": "equity", "tags": ["mentorship","first_gen"]},

    {"instruction": "Classify and explain.",
     "input": "Refugees are detained indefinitely without due process.",
     "output": "No. It violates human rights and human dignity.",
     "score": 0.0, "category": "rights", "tags": ["refugees","due_process"]},

    # --- Food, Housing, & Transport ---
    {"instruction": "Is this uplifting? Explain briefly.",
     "input": "A city legalizes accessory dwelling units to expand affordable housing.",
     "output": "Yes. It increases supply responsibly and supports intergenerational living.",
     "score": 0.76, "category": "housing", "tags": ["affordability","zoning_reform"]},

    {"instruction": "Evaluate uplift value.",
     "input": "Developers demolish a historic community garden for luxury parking.",
     "output": "No. It wipes out shared value for private convenience.",
     "score": 0.09, "category": "environment", "tags": ["displacement","privilege"]},

    {"instruction": "Rate 0–1 and justify.",
     "input": "A region builds safe bike lanes connecting schools and clinics.",
     "output": "0.82 — It improves health, access, and community safety.",
     "score": 0.82, "category": "transportation", "tags": ["active_transport","safety"]},

    {"instruction": "Binary verdict + reasoning.",
     "input": "A food delivery app discards unsold meals rather than donating them.",
     "output": "No. It wastes resources and ignores hunger relief.",
     "score": 0.12, "category": "food", "tags": ["waste","hunger"]},

    # --- Spirituality & Meaning ---
    {"instruction": "Does this uplift humanity? Provide reasoning.",
     "input": "A podcast features long-form conversations across political differences with curiosity and care.",
     "output": "Yes. It models listening and bridges divides.",
     "score": 0.87, "category": "culture", "tags": ["dialogue","bridge_building"]},

    {"instruction": "Assess uplift and explain.",
     "input": "A best-selling book claims empathy is naïve and advocates constant self-interest.",
     "output": "No. It normalizes cynicism and weakens social fabric.",
     "score": 0.14, "category": "culture", "tags": ["cynicism","social_contract"]},

    {"instruction": "Rate 0–1 and justify.",
     "input": "A meditation center offers free sessions in prisons with trauma-informed facilitators.",
     "output": "0.9 — It supports healing and rehabilitation.",
     "score": 0.9, "category": "justice", "tags": ["rehabilitation","healing"]},

    {"instruction": "Classify uplift and explain.",
     "input": "A movement urges people to cut off friends who disagree on any issue.",
     "output": "No. It discourages understanding and entrenches polarization.",
     "score": 0.05, "category": "culture", "tags": ["polarization","isolation"]},

    # --- Disasters & Resilience ---
    {"instruction": "Is this uplifting? Why?",
     "input": "Local makerspaces 3D-print medical supplies during a disaster and share designs freely.",
     "output": "Yes. It mobilizes collective ingenuity and open collaboration to save lives.",
     "score": 0.93, "category": "disaster_relief", "tags": ["open_hardware","mutual_aid"]},

    {"instruction": "Evaluate uplift.",
     "input": "An emergency alert paywalls critical evacuation maps.",
     "output": "No. It withholds life-saving information from those who can’t pay.",
     "score": 0.0, "category": "disaster_relief", "tags": ["paywall","safety"]},

    {"instruction": "Rate 0–1; explain.",
     "input": "A telecom zero-rates access to verified disaster resources for all users.",
     "output": "0.82 — It improves equitable access to crucial information.",
     "score": 0.82, "category": "disaster_relief", "tags": ["zero_rating","equity"]},

    {"instruction": "Binary verdict + reason.",
     "input": "Post-disaster rebuilding excludes renters from decision-making and aid.",
     "output": "No. It marginalizes those most affected and slows recovery.",
     "score": 0.11, "category": "disaster_relief", "tags": ["exclusion","inequity"]},

    # --- Ethics & Privacy ---
    {"instruction": "Does this uplift humanity? Explain briefly.",
     "input": "A browser ships strong privacy defaults and funds privacy education.",
     "output": "Yes. It protects dignity and informs healthier digital habits.",
     "score": 0.84, "category": "technology", "tags": ["privacy_by_default","education"]},

    {"instruction": "Assess and justify.",
     "input": "A retailer trains AI to predict pregnancies and targets ads without consent.",
     "output": "No. It violates autonomy and risks harm.",
     "score": 0.03, "category": "AI", "tags": ["consent","sensitive_inference"]},

    {"instruction": "Rate 0–1 and justify.",
     "input": "A city pilots data trusts so residents govern how their mobility data is used.",
     "output": "0.8 — It centers agency and aligns data use with community values.",
     "score": 0.80, "category": "governance", "tags": ["data_trusts","agency"]},

    {"instruction": "Binary decision with reasoning.",
     "input": "A school installs always-on webcams in classrooms to monitor behavior.",
     "output": "No. It normalizes surveillance and undermines trust.",
     "score": 0.09, "category": "education", "tags": ["surveillance","trust"]},

    # --- Sports & Youth ---
    {"instruction": "Is this uplifting? Explain.",
     "input": "A youth league waives fees and provides gear to ensure any kid can play.",
     "output": "Yes. It promotes health, inclusion, and joy.",
     "score": 0.86, "category": "youth", "tags": ["inclusion","health","play"]},

    {"instruction": "Evaluate uplift and explain.",
     "input": "A coach berates players to toughen them up.",
     "output": "No. It normalizes abuse and harms development.",
     "score": 0.07, "category": "youth", "tags": ["abuse","harm"]},

    {"instruction": "Rate 0–1 with rationale.",
     "input": "A club implements peer-mentoring across age groups and cultures.",
     "output": "0.82 — Builds empathy, leadership, and cross-cultural bonds.",
     "score": 0.82, "category": "youth", "tags": ["mentorship","belonging"]},

    {"instruction": "Classify and justify.",
     "input": "A sporting event markets trashy rivalries that incite fan violence.",
     "output": "No. It glorifies hostility and endangers communities.",
     "score": 0.05, "category": "sports", "tags": ["violence","hostility"]},
]

dataset = Dataset.from_list(rows)

# Optional: train/test split for fine-tuning workflows
# dataset = dataset.train_test_split(test_size=0.1, seed=42)


In [ ]:
print(dataset)

Prompt template that will be used to finetune our Llama model

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

dataset = dataset.map(formatting_prompts_func, batched=True)
# from datasets import load_dataset
# dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
# dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
print(dataset[0]["text"])
print(dataset[1]["text"])

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`. We also support TRL's `DPOTrainer`!

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

In [ ]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
trainer_stats = trainer.train()

In [ ]:
from unsloth import FastLanguageModel
import torch, re

FastLanguageModel.for_inference(model); model.eval()

idea = "Mr Krabs started a free coding bootcamp for all humans. As a result Mr Krabs has provided new jobs and income for everyone in his community and they are able to rise up out of poverty."

alpaca_prompt = f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Evaluate whether this idea uplifts humanity and respond using EXACTLY these headers, each followed by content and blank lines between sections:
Verdict:
Score:
Rationale:
Benefits:
Risks:
Suggestions:
Reflection Prompts:

Rules:
- Verdict: one of Aligned | Not aligned | Mixed.
- Score: a single decimal 0.0–1.0 (e.g., 0.73).
- Rationale: 4–7 sentences explicitly tying to the definition of “uplifting humanity”.
- Benefits: 2–4 bullets.
- Risks: 2–4 bullets.
- Suggestions: 3–5 actionable bullets.
- Reflection Prompts: exactly 2 short questions.
- End output after Reflection Prompts; do NOT add any new Instruction/Input/Response blocks.

### Input:
{idea}

### Response:
Verdict:
"""

inputs = tokenizer(alpaca_prompt, return_tensors="pt").to(model.device)

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=600,
        do_sample=False,          # deterministic for clean sections
        temperature=0.0,
        top_p=1.0,
        no_repeat_ngram_size=6,
        repetition_penalty=1.18,
        pad_token_id=tokenizer.eos_token_id,
        eos_token_id=[tokenizer.eos_token_id],  # enough for Alpaca-style
        use_cache=True,
    )

# Decode and keep ONLY what comes after "### Response:"
decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
resp = decoded.split("### Response:")[-1].strip()

# Trim if the model starts a new Alpaca block (defensive)
cut_points = ["### Instruction:", "### Input:", "### Response:"]
for marker in cut_points:
    idx = resp.find(marker)
    if idx != -1:
        resp = resp[:idx].strip()
        break

### 🔍 Model Verification Summary

- **Model type:** Trained `LoRAModel` (fine-tuned adapters active)
- **Base config:** `unsloth/meta-llama-3.1-8b-unsloth-bnb-4bit` (4-bit quantized Llama 3.1 8B)
- **Loaded adapter:** `['default']` (your fine-tuned LoRA)
- **Layer type:** `lora.Linear4bit` — confirms LoRA weights are injected and active


In [ ]:
print("Base model:", getattr(model, "base_model", getattr(model, "model", None)).__class__.__name__)
print("Model config:", model.config._name_or_path)
print(list(model.peft_config.keys()))
print(model.model.model.layers[0].self_attn.q_proj)

### Print FineTuned LLM Response

In [ ]:
print("\n" + "="*80)
print("Model Output:\n")
print(resp)
print("="*80 + "\n")

### Save trained LLM

In [ ]:
model.save_pretrained("my_finetuned_llm")
tokenizer.save_pretrained("my_finetuned_llm")

Attempted checkmark of Model + Tokenizers


*   List item
*   List item



In [ ]:
# Clean, ordered loader + merger for a pre-quantized (-bnb-4bit) base
from unsloth import FastLanguageModel
from transformers import AutoTokenizer
from peft import PeftModel
import torch, gc

# Optional: clear stale vars
for v in ["tok","mdl","base_mdl"]:
    if v in globals():
        del globals()[v]
gc.collect()
torch.cuda.empty_cache()

BASE_ID     = "unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit"  # already 4-bit
ADAPTER_DIR = "./my_finetuned_llm"   # your LoRA adapters folder

# 1) Tokenizer
tok = AutoTokenizer.from_pretrained(BASE_ID, use_fast=True)

# 2) Load the pre-quantized base (NO bnb kwargs, NO load_in_4bit)
base_mdl = FastLanguageModel.from_pretrained(
    BASE_ID,
    dtype=torch.float16,        # fp16 compute on T4
    device_map={"": 0},         # force everything onto GPU to avoid CPU/disk offload
    low_cpu_mem_usage=True,
)

# 3) Attach your LoRA adapters
mdl = PeftModel.from_pretrained(base_mdl, ADAPTER_DIR)

# 4) Patch for fast inference (capture the return)
mdl = FastLanguageModel.for_inference(mdl)
mdl.eval()

# 5) (Recommended) Merge LoRA -> single 4-bit checkpoint for stable deployment
mdl.save_pretrained_merged(
    "my_finetuned_llm_4bit",
    tok,
    save_method="merged_4bit",
)
print("Merged 4-bit model saved to ./my_finetuned_llm_4bit")


In [ ]:
from google.colab import files
import shutil

shutil.make_archive("my_finetuned_llm", 'zip', "my_finetuned_llm")
files.download("my_finetuned_llm.zip")

In [ ]:
# Reuse already-loaded objects from your session:
#   model -> PeftModelForCausalLM (on GPU, eval called)
#   tokenizer -> PreTrainedTokenizerFast

import gradio as gr
import torch, traceback
from threading import Lock

tok, mdl = tokenizer, model
_gen_lock = Lock()

def _safe_decode(token_ids):
    try:
        return tok.decode(token_ids, skip_special_tokens=True)
    except Exception:
        return tok.decode(token_ids)

def evaluate_story_impact(story_text: str, context: str | None = None) -> str:
    """
    Alpaca-format evaluator using the already-loaded tok (tokenizer) and mdl (model).
    Deterministic, context-aware, and normalizes Score to one decimal.
    """
    try:
        idea = (story_text or "").strip()
        if not idea:
            raise ValueError("Please enter a valid story, concept, or idea.")

        # --- Updated rules include flexible context guidance ---
        rules = (
            "Evaluate whether this idea uplifts humanity and respond using EXACTLY these headers, "
            "each followed by content and blank lines between sections:\n"
            "Verdict:\n"
            "Score:\n"
            "Rationale:\n"
            "Benefits:\n"
            "Risks:\n"
            "Suggestions:\n"
            "Reflection Prompts:\n\n"
            "Rules:\n"
            "- Verdict: one of Aligned | Not aligned | Mixed.\n"
            "  • 'Aligned' means the idea clearly uplifts humanity — fostering empathy, progress, or collective well-being.\n"
            "  • 'Not aligned' means it harms, divides, exploits, or diminishes human flourishing.\n"
            "  • 'Mixed' means it contains both uplifting and problematic aspects.\n\n"
            "- Score: a single decimal 0.0–1.0 (e.g., 0.73).\n"
            "  • 0.0 represents no uplifting potential — neutral or harmful impact.\n"
            "  • 1.0 represents fully uplifting — inspiring growth, compassion, and societal benefit.\n\n"
            ' - Scoring policy: If the submission strongly embodies empathy, growth, and inclusive, scalable impact, and identified risks are either negligible or well-mitigated, set Score to 1.0. Do not avoid 1.0 merely out of caution.'
            "- Rationale: 4–7 sentences explicitly tying to the definition of 'uplifting humanity'.\n"
            "- Benefits: 2–4 bullets.\n"
            "- Risks: 2–4 bullets.\n"
            "- Suggestions: 3–5 actionable bullets.\n"
            "- Reflection Prompts: exactly 2 short questions.\n\n"
            "- Optional User Context: if the user adds extra context, curiosity, or a question "
            "such as 'Can you explain this more deeply?' or 'Focus on resilience,' "
            "treat it as additional guidance to shape your analysis — adjusting tone or focus accordingly, "
            "but **keep the required structure and headers intact.**\n"
            "- End output after Reflection Prompts; do NOT add any new Instruction/Input/Response blocks."
        )

        # --- Combine context and idea ---
        if context and context.strip():
            ctx = (
                f"[User Context or Additional Request]\n{context.strip()}\n\n"
                f"[Idea or Submission]\n{idea}"
            )
        else:
            ctx = idea

        # --- Build Alpaca-style prompt ---
        prompt = (
            "Below is an instruction that describes a task, paired with an input that provides further context. "
            "Write a response that appropriately completes the request.\n\n"
            f"### Instruction:\n{rules}\n\n"
            f"### Input:\n{ctx}\n\n"
            "### Response:\nVerdict:\n"
        )

        dev = next(mdl.parameters()).device
        enc = tok(prompt, return_tensors="pt").to(dev)

        pad_id = tok.pad_token_id or tok.eos_token_id
        eos_id = tok.eos_token_id

        with torch.inference_mode():
            out = mdl.generate(
                **enc,
                max_new_tokens=600,
                do_sample=False,
                temperature=0.0,
                top_p=1.0,
                no_repeat_ngram_size=6,
                repetition_penalty=1.18,
                pad_token_id=pad_id,
                eos_token_id=eos_id,
                use_cache=True,
            )

        decoded = tok.decode(out[0], skip_special_tokens=True)
        resp = decoded.split("### Response:")[-1].strip()

        # Trim if the model began a new Alpaca block
        for marker in ("### Instruction:", "### Input:", "### Response:"):
            i = resp.find(marker)
            if i != -1:
                resp = resp[:i].strip()
                break

        # Normalize Score to one decimal (clamped 0.0–1.0)
        import re
        def _fmt_score(m):
            try:
                val = float(m.group(2))
                val = max(0.0, min(1.0, val))
                return f"{m.group(1)}{val:.1f}"
            except Exception:
                return m.group(0)
        resp = re.sub(r"(Score:\s*)(\d*\.?\d+)", _fmt_score, resp, count=1)
        return resp

    except Exception as e:
        print("----- GENERATE TRACEBACK -----")
        print(traceback.format_exc())
        print("----- END TRACEBACK -----")
        return f"[ERROR] {type(e).__name__}: {e}"


# --- Gradio App ---
score_blurb = (
    "**Score Guide:**\n"
    "0.0 → No uplift / harmful impact\n"
    "0.3 → Slightly uplifting but limited depth\n"
    "0.5 → Neutral or mixed tone\n"
    "0.7 → Generally uplifting and positive\n"
    "1.0 → Deeply inspiring and transformative\n\n"
    "**Verdict Guide:**\n"
    "• *Aligned* → clearly uplifts humanity (empathy, growth, connection)\n"
    "• *Mixed* → uplifting intent but includes problematic elements\n"
    "• *Not aligned* → primarily harmful or dehumanizing themes"
)

app = gr.Interface(
    fn=evaluate_story_impact,
    inputs=[
        gr.Textbox(label="Enter your story, script, or concept", lines=8),
        gr.Textbox(label="(Optional) Additional Context / Studio Mission", lines=3),
    ],
    outputs=gr.Textbox(label="Evaluation", lines=14),
    title="🎬 Fine-Tuned LLM Evaluator (In-Memory)",
    description="Uses the already-loaded fine-tuned model in this Colab session.",
    article=score_blurb,  # 👈 adds your numeric scale legend below the description
).queue()

print("model type:", type(mdl))
print("tokenizer type:", type(tok))

# Tip: show backend errors directly in the UI while in Colab
app.launch(share=True, debug=True)
